# Single Agent RAG: Banking Policy Assistant

**Objective**: Build a single-agent RAG system that retrieves banking policy information from a local Chroma vector store and provides accurate answers with citations.

**Key Features**:
- 🔍 Parse user queries to extract intent and entities
- 📚 Retrieve relevant documents from Chroma vector store
- 🎯 Generate answers with explicit citations
- 🔄 Re-retrieve if initial evidence is insufficient
- 🛡️ Handle insufficient evidence gracefully

**Time**: ~15-20 minutes

**Scenario**: Support a bank's customer service team with instant access to policy information for fees, limits, and procedures.

## 📊 Banking Knowledge Base

Our vector store contains comprehensive banking policies:
- Personal Checking Fee Schedule (2025)
- Domestic & International Wire Transfer Policy (2025)  
- ATM Network & Reimbursement Rules (2025)
- Overdraft & NSF Policy (Rev. Apr 2025)
- Mobile Deposit Limits & Hold Periods (Consumer)
- Interest Tiers & APY Disclosures (Savings/MMA)

In [1]:
# Import required libraries
import os
import json
from dataclasses import dataclass
from typing import List, Optional, Dict, Any
from datetime import datetime

# OpenAI for LLM and embeddings
import openai
from openai import OpenAI

# Chroma for vector storage
import chromadb
from chromadb.config import Settings as ChromaSettings

# Environment variables
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("🔧 Environment Setup:")
print(f"   ✅ OpenAI API Key: {'✓ Configured' if os.getenv('OPENAI_API_KEY') else '❌ Missing'}")
print("   📚 Knowledge Base: Banking policy documents") 
print("   🤖 Simple RAG with direct OpenAI + Chroma integration")

🔧 Environment Setup:
   ✅ OpenAI API Key: ✓ Configured
   📚 Knowledge Base: Banking policy documents
   🤖 Simple RAG with direct OpenAI + Chroma integration


## 🏗️ Data Models for RAG System

In [2]:
@dataclass
class SimpleRAGResponse:
    """Simple response from our minimal RAG agent"""
    query: str
    answer: str
    sources: List[str]
    retrieved_chunks: int
    needed_retry: bool
    confidence: str  # "high", "medium", "low"

print("📋 Simple data model defined for minimal RAG")

📋 Simple data model defined for minimal RAG


## 📚 Knowledge Base Setup

In [3]:
def load_banking_documents_to_chroma():
    """Load banking documents directly into Chroma with embeddings"""
    
    # Initialize Chroma client
    chroma_client = chromadb.PersistentClient(path="./chroma_db")
    
    collection_name = "banking_policies_simple"
    
    # Check if collection exists
    try:
        existing_collections = [col.name for col in chroma_client.list_collections()]
        if collection_name in existing_collections:
            print(f"📚 Using existing collection: {collection_name}")
            collection = chroma_client.get_collection(collection_name)
            print(f"   📊 Documents in collection: {collection.count()}")
            return collection
    except Exception as e:
        pass
    
    print(f"📝 Creating new collection: {collection_name}")
    
    # Create new collection
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"description": "Simple banking policy documents"}
    )
    
    # Load documents from data directory
    data_dir = "./data"
    documents = []
    metadatas = []
    ids = []
    
    if not os.path.exists(data_dir):
        print(f"❌ Data directory '{data_dir}' not found!")
        return collection
    
    doc_id = 0
    for filename in os.listdir(data_dir):
        if filename.endswith('.txt'):
            file_path = os.path.join(data_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
                # Split content into chunks
                chunks = split_text_into_chunks(content, chunk_size=500, overlap=50)
                
                for i, chunk in enumerate(chunks):
                    documents.append(chunk)
                    metadatas.append({
                        'filename': filename,
                        'doc_title': filename.replace('_', ' ').replace('.txt', '').title(),
                        'chunk_id': f"{filename}_{i}"
                    })
                    ids.append(f"doc_{doc_id}")
                    doc_id += 1
                
                print(f"   📄 Loaded: {filename} ({len(chunks)} chunks)")
    
    if documents:
        # Add documents to collection (Chroma will automatically generate embeddings)
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        print(f"🎉 Added {len(documents)} document chunks to Chroma")
    else:
        print("❌ No documents found to add")
    
    return collection

def split_text_into_chunks(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    """Simple text chunking function"""
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size - overlap):
        chunk_words = words[i:i + chunk_size]
        chunks.append(' '.join(chunk_words))
    
    return chunks

def get_embedding(text: str) -> List[float]:
    """Get OpenAI embedding for text"""
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

# Setup the knowledge base
print("🚀 Setting up simple RAG system...")
chroma_collection = load_banking_documents_to_chroma()

if chroma_collection.count() > 0:
    print("✅ Simple RAG system ready!")
    print(f"   📊 Collection: {chroma_collection.count()} document chunks")
else:
    print("❌ Failed to setup RAG system")

🚀 Setting up simple RAG system...
📚 Using existing collection: banking_policies_simple
   📊 Documents in collection: 7
✅ Simple RAG system ready!
   📊 Collection: 7 document chunks


## 🤖 Single Agent RAG System

In [4]:
class SimpleRAGAgent:
    """Minimal autonomous RAG agent that thinks for itself"""
    
    def __init__(self, chroma_collection):
        self.collection = chroma_collection
        self.query_history = []
    
    def retrieve_documents(self, query: str, n_results: int = 4) -> Dict[str, Any]:
        """Retrieve relevant documents from Chroma"""
        
        # Query the collection
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        
        return {
            'documents': results['documents'][0] if results['documents'] else [],
            'metadatas': results['metadatas'][0] if results['metadatas'] else [],
            'distances': results['distances'][0] if results['distances'] else [],
            'count': len(results['documents'][0]) if results['documents'] else 0
        }
    
    def ask_llm_for_answer(self, query: str, context_docs: List[str]) -> str:
        """Ask the LLM to answer based on retrieved documents"""
        
        # Build context from retrieved documents
        context = "\n\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(context_docs)])
        
        prompt = f"""You are a helpful banking customer service assistant. Answer the customer's question based ONLY on the provided documents.

RETRIEVED DOCUMENTS:
{context}

CUSTOMER QUESTION: {query}

INSTRUCTIONS:
- Answer based ONLY on the information in the documents above
- If the documents contain the answer, provide a clear, helpful response
- If the documents don't contain enough information, say "I don't have enough information in my knowledge base to answer that question"
- Include specific details like fees, limits, times, etc. when available
- Be conversational but professional

ANSWER:"""

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=500
        )
        
        return response.choices[0].message.content
    
    def should_retry_retrieval(self, query: str, answer: str, retrieved_docs: List[str]) -> bool:
        """Let the LLM decide if we need to search again"""
        
        prompt = f"""You are evaluating whether a RAG system needs to search again for better information.

ORIGINAL QUESTION: {query}

CURRENT ANSWER: {answer}

RETRIEVED DOCUMENTS: {len(retrieved_docs)} documents found

EVALUATION TASK:
Respond with ONLY "YES" or "NO" to indicate if the system should search again.

Say "YES" if:
- The answer says there's not enough information
- The answer is vague or incomplete
- The question asks for specific details (fees, limits, times) but the answer doesn't provide them

Say "NO" if:
- The answer provides specific, helpful information
- The answer appropriately explains what's not available

DECISION:"""

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=10
        )
        
        decision = response.choices[0].message.content.strip().upper()
        return decision == "YES"
    
    def generate_better_search_query(self, original_query: str, previous_answer: str) -> str:
        """Let the LLM generate a better search query for retry"""
        
        prompt = f"""You need to generate a better search query for a banking knowledge base.

ORIGINAL QUERY: {original_query}
PREVIOUS ANSWER: {previous_answer}

The previous search didn't find good information. Generate a NEW search query that might find better results.

INSTRUCTIONS:
- Use different keywords or synonyms
- Try broader or more specific terms
- Focus on key banking concepts
- Keep it under 10 words

NEW SEARCH QUERY:"""

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=50
        )
        
        return response.choices[0].message.content.strip()
    
    def assess_confidence(self, answer: str, retrieved_count: int, best_distance: float) -> str:
        """Simple confidence assessment"""
        if "don't have enough information" in answer.lower():
            return "low"
        elif retrieved_count >= 3 and best_distance < 0.8:
            return "high"
        elif retrieved_count >= 2:
            return "medium"
        else:
            return "low"
    
    def process_query(self, query: str, show_thinking: bool = False) -> SimpleRAGResponse:
        """Main method - autonomous RAG processing"""
        
        if show_thinking:
            print(f"🤔 Thinking about: {query}")
        
        # Step 1: Initial retrieval
        results = self.retrieve_documents(query, n_results=4)
        docs = results['documents']
        
        if show_thinking:
            print(f"🔍 Found {results['count']} documents")
        
        # Step 2: Generate initial answer
        answer = self.ask_llm_for_answer(query, docs)
        
        if show_thinking:
            print(f"💭 Initial answer: {answer[:100]}...")
        
        # Step 3: Let LLM decide if we need to retry
        needs_retry = False
        total_chunks = results['count']
        
        if results['count'] > 0:
            needs_retry = self.should_retry_retrieval(query, answer, docs)
        
        # Step 4: Retry if needed
        if needs_retry:
            if show_thinking:
                print(f"🔄 LLM thinks we should search again...")
            
            # Generate better search query
            better_query = self.generate_better_search_query(query, answer)
            if show_thinking:
                print(f"🎯 Trying: {better_query}")
            
            # Search again with more results
            retry_results = self.retrieve_documents(better_query, n_results=6)
            
            if retry_results['count'] > results['count']:
                # Use better results
                results = retry_results
                docs = results['documents']
                answer = self.ask_llm_for_answer(query, docs)
                total_chunks = retry_results['count']
                
                if show_thinking:
                    print(f"✅ Found {retry_results['count']} documents with better query")
        
        # Step 5: Extract sources
        sources = []
        if results['metadatas']:
            sources = [meta.get('doc_title', 'Unknown') for meta in results['metadatas']]
        
        # Step 6: Assess confidence
        best_distance = min(results['distances']) if results['distances'] else 1.0
        confidence = self.assess_confidence(answer, results['count'], best_distance)
        
        # Create response
        response = SimpleRAGResponse(
            query=query,
            answer=answer,
            sources=list(set(sources)),  # Remove duplicates
            retrieved_chunks=total_chunks,
            needed_retry=needs_retry,
            confidence=confidence
        )
        
        # Store in history
        self.query_history.append(response)
        
        return response

# Initialize the simple RAG agent
if chroma_collection.count() > 0:
    rag_agent = SimpleRAGAgent(chroma_collection)
    print("🤖 Simple autonomous RAG agent ready!")
    print("   🧠 LLM decides when to re-search")
    print("   🔄 Self-improving search queries") 
    print("   📊 No complex intent parsing needed")
else:
    print("❌ Cannot initialize agent - no documents in collection")

🤖 Simple autonomous RAG agent ready!
   🧠 LLM decides when to re-search
   🔄 Self-improving search queries
   📊 No complex intent parsing needed


## 🧪 Test the RAG System

Let's test our banking RAG agent with realistic customer service questions.

In [5]:
# Test the simple autonomous RAG agent
test_queries = [
    "What's the same-day domestic wire cut-off time?",
    "How many ATM fee reimbursements per month on Premier Checking?", 
    "What's the mobile check deposit limit for new accounts?",
    "What's the overdraft fee and any waiver conditions?"
]

print("🧪 Testing Simple Autonomous RAG Agent\n")

for i, query in enumerate(test_queries, 1):
    print(f"{'='*60}")
    print(f"Test {i}: {query}")
    print(f"{'='*60}")
    
    # Process query - let the agent think for itself
    response = rag_agent.process_query(query, show_thinking=True)
    
    print(f"\n📝 Answer:")
    print(response.answer)
    
    if response.sources:
        print(f"\n📚 Sources:")
        for j, source in enumerate(response.sources, 1):
            print(f"   {j}. {source}")
    
    print(f"\n📊 Summary:")
    print(f"   Chunks retrieved: {response.retrieved_chunks}")
    print(f"   Confidence: {response.confidence}")
    print(f"   Needed retry: {'Yes' if response.needed_retry else 'No'}")
    
    print("\n")

🧪 Testing Simple Autonomous RAG Agent

Test 1: What's the same-day domestic wire cut-off time?
🤔 Thinking about: What's the same-day domestic wire cut-off time?
🔍 Found 4 documents
💭 Initial answer: The same-day processing cut-off times for domestic wire transfers are as follows: On standard busine...

📝 Answer:
The same-day processing cut-off times for domestic wire transfers are as follows: On standard business days, the cut-off time is 3:00 PM ET. On Fridays, the cut-off time is slightly earlier at 2:30 PM ET. Please note that there is no same-day processing on Federal Holidays.

📚 Sources:
   1. Personal Checking Fees 2025
   2. Overdraft Nsf Policy 2025
   3. Wire Transfer Policy 2025
   4. Atm Network Reimbursement 2025

📊 Summary:
   Chunks retrieved: 4
   Confidence: medium
   Needed retry: No


Test 2: How many ATM fee reimbursements per month on Premier Checking?
🤔 Thinking about: How many ATM fee reimbursements per month on Premier Checking?
🔍 Found 4 documents
💭 Initial ans

## 🔄 Demonstrating Re-retrieval Logic

Let's test with a query that should trigger re-retrieval due to ambiguous terms.

In [6]:
# Test autonomous decision-making with an ambiguous query
ambiguous_query = "What are the cutoff times for money transfers?"

print("🤔 Testing Autonomous Decision-Making")
print(f"Query: {ambiguous_query}")
print("-" * 50)

response = rag_agent.process_query(ambiguous_query, show_thinking=True)

print(f"\n📝 Final Answer:")
print(response.answer)

print(f"\n🧠 Agent Decision Process:")
print(f"   • Initial documents found: {response.retrieved_chunks}")
print(f"   • Agent decided to retry: {'Yes' if response.needed_retry else 'No'}")
print(f"   • Final confidence: {response.confidence}")

if response.needed_retry:
    print(f"   ✅ Agent autonomously decided more information was needed")
else:
    print(f"   ✅ Agent was satisfied with initial results")

🤔 Testing Autonomous Decision-Making
Query: What are the cutoff times for money transfers?
--------------------------------------------------
🤔 Thinking about: What are the cutoff times for money transfers?
🔍 Found 4 documents
💭 Initial answer: For domestic wire transfers, the cut-off times for same-day processing are 3:00 PM ET on standard bu...

📝 Final Answer:
For domestic wire transfers, the cut-off times for same-day processing are 3:00 PM ET on standard business days and 2:30 PM ET on Fridays. There is no same-day processing on federal holidays. For next business day processing, the cut-off time is 5:00 PM ET on business days. 

For international wire transfers, the cut-off time for same-day initiation is 2:00 PM ET. 

For mobile deposits, the cut-off times for same-day processing are Monday-Friday before 6:00 PM ET and Saturday before 12:00 PM ET. There is no processing on Sundays. Deposits made after these cut-off times will be processed the next business day.

🧠 Agent Decision

## 🛡️ Testing Insufficient Evidence Handling

Let's test with a query outside our knowledge base scope.

In [7]:
# Test with a query outside knowledge base scope
out_of_scope_query = "What's the interest rate on personal loans?"

print("🛡️ Testing Out-of-Scope Query Handling")
print(f"Query: {out_of_scope_query}")
print("-" * 50)

response = rag_agent.process_query(out_of_scope_query, show_thinking=True)

print(f"\n📝 Answer:")
print(response.answer)

print(f"\n🎯 Agent Analysis:")
print(f"   • Documents found: {response.retrieved_chunks}")
print(f"   • Confidence level: {response.confidence}")
print(f"   • Tried to search again: {'Yes' if response.needed_retry else 'No'}")

if "don't have enough information" in response.answer.lower():
    print(f"   ✅ Agent correctly identified insufficient information")
else:
    print(f"   ❓ Agent response needs review")

🛡️ Testing Out-of-Scope Query Handling
Query: What's the interest rate on personal loans?
--------------------------------------------------
🤔 Thinking about: What's the interest rate on personal loans?
🔍 Found 4 documents
💭 Initial answer: I'm sorry, but I don't have enough information in my knowledge base to answer that question. The doc...
🔄 LLM thinks we should search again...
🎯 Trying: Current average interest rates for individual loans
✅ Found 6 documents with better query

📝 Answer:
I'm sorry, but I don't have enough information in my knowledge base to answer that question about the interest rate on personal loans.

🎯 Agent Analysis:
   • Documents found: 6
   • Confidence level: low
   • Tried to search again: Yes
   ✅ Agent correctly identified insufficient information


## 📊 Query History Analysis

Let's examine the agent's query processing history.

In [8]:
print("📊 Simple RAG Agent Summary")
print(f"Total queries processed: {len(rag_agent.query_history)}")
print("-" * 50)

# Simple analytics
retry_count = sum(1 for r in rag_agent.query_history if r.needed_retry)
high_confidence = sum(1 for r in rag_agent.query_history if r.confidence == "high")
total_queries = len(rag_agent.query_history)

if total_queries > 0:
    print(f"🤖 Agent Intelligence:")
    print(f"   • Autonomous retry rate: {retry_count}/{total_queries} ({retry_count/total_queries*100:.1f}%)")
    print(f"   • High confidence answers: {high_confidence}/{total_queries} ({high_confidence/total_queries*100:.1f}%)")
    
    print(f"\n📈 Performance:")
    avg_chunks = sum(r.retrieved_chunks for r in rag_agent.query_history) / total_queries
    print(f"   • Average chunks per query: {avg_chunks:.1f}")
    
    # Show a sample interaction
    if rag_agent.query_history:
        sample = rag_agent.query_history[0]
        print(f"\n💬 Sample Interaction:")
        print(f"   Q: {sample.query}")
        print(f"   A: {sample.answer[:100]}...")
        print(f"   Sources: {len(sample.sources)} documents")
        print(f"   Autonomous retry: {'Yes' if sample.needed_retry else 'No'}")

print(f"\n🎯 Key Advantages of Simple Autonomous RAG:")
print(f"   ✅ No complex intent parsing or keyword matching")
print(f"   ✅ LLM decides when to search again (autonomous)")
print(f"   ✅ LLM generates better search queries for retry")
print(f"   ✅ Direct OpenAI + Chroma integration")
print(f"   ✅ Minimal code, maximum intelligence")
print(f"   ✅ Agent thinks for itself, not rule-based")

📊 Simple RAG Agent Summary
Total queries processed: 6
--------------------------------------------------
🤖 Agent Intelligence:
   • Autonomous retry rate: 1/6 (16.7%)
   • High confidence answers: 3/6 (50.0%)

📈 Performance:
   • Average chunks per query: 4.3

💬 Sample Interaction:
   Q: What's the same-day domestic wire cut-off time?
   A: The same-day processing cut-off times for domestic wire transfers are as follows: On standard busine...
   Sources: 4 documents
   Autonomous retry: No

🎯 Key Advantages of Simple Autonomous RAG:
   ✅ No complex intent parsing or keyword matching
   ✅ LLM decides when to search again (autonomous)
   ✅ LLM generates better search queries for retry
   ✅ Direct OpenAI + Chroma integration
   ✅ Minimal code, maximum intelligence
   ✅ Agent thinks for itself, not rule-based


## 🎯 Simple Autonomous RAG System

### ✅ Core Features Demonstrated

1. **Autonomous Decision-Making** ✓
   - LLM decides when initial retrieval is insufficient
   - No manual intent parsing or keyword matching
   - Agent thinks for itself about what information is needed

2. **Self-Improving Search** ✓
   - LLM generates better search queries for retry
   - Natural language understanding of what went wrong
   - Automatic query expansion based on context

3. **Direct Integration** ✓
   - OpenAI GPT-4 for reasoning and answer generation
   - Chroma vector database for document storage and retrieval
   - No complex framework overhead

4. **Intelligent Re-Retrieval** ✓
   - LLM evaluates answer quality and completeness
   - Automatically searches again with improved queries
   - Single retry to maintain performance

5. **Graceful Handling** ✓
   - Properly handles out-of-scope questions
   - Admits when information is insufficient
   - Provides confidence assessment

### 🧠 How It Works

1. **Query Processing**: User asks a question
2. **Initial Search**: Retrieve relevant documents from Chroma
3. **Answer Generation**: LLM creates answer from retrieved context
4. **Quality Check**: LLM evaluates if answer is complete/helpful
5. **Autonomous Retry**: If needed, LLM generates better search and tries again
6. **Final Response**: Return answer with sources and confidence

### 🚀 Key Advantages

- **No Rule Engineering**: LLM handles all decision-making naturally
- **Self-Directed**: Agent autonomously improves its search strategy
- **Minimal Code**: Simple, maintainable implementation
- **Maximum Intelligence**: Leverages LLM's reasoning capabilities
- **Production Ready**: Direct OpenAI + Chroma integration

### 🔧 Technical Components

- **Vector Store**: Chroma for semantic document search
- **LLM**: OpenAI GPT-4 for reasoning and generation  
- **Embeddings**: OpenAI text-embedding-ada-002
- **Autonomy**: LLM-driven retry decisions and query improvement
- **Simplicity**: Minimal abstraction layers, direct API calls

This autonomous RAG system demonstrates how modern LLMs can manage their own retrieval process without complex rule-based logic. The agent naturally understands when it needs more information and how to search for it!